In [1]:
!wget -O archive.zip https://www.dropbox.com/s/5yjv2hth42wbjz3/archive.zip?dl=0
!unzip "archive.zip"

--2021-02-09 22:00:11--  https://www.dropbox.com/s/5yjv2hth42wbjz3/archive.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6027:18::a27d:4812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/5yjv2hth42wbjz3/archive.zip [following]
--2021-02-09 22:00:12--  https://www.dropbox.com/s/raw/5yjv2hth42wbjz3/archive.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd3c225842e8544e42a1539ae9d.dl.dropboxusercontent.com/cd/0/inline/BIpBwMrH0SXp51lU6jnI5LxaHCFcl7TrakalYIKIU6spAfWKjNZHA4oREtFo4_CSsaXQCOddph8gmRMTG9LKGjz8umqO1ij_eNEXNbAGyTI0Hbzvpmzu8ABtaDhemBEaFk0/file# [following]
--2021-02-09 22:00:12--  https://ucd3c225842e8544e42a1539ae9d.dl.dropboxusercontent.com/cd/0/inline/BIpBwMrH0SXp51lU6jnI5LxaHCFcl7TrakalYIKIU6spAfWKjNZHA4oREtFo4_CSsaXQCOddph8gmRMTG9LKGjz8umqO1ij_eNEXNbAGyT

In [44]:
import pandas as pd
import numpy as np
import string 
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import re
%matplotlib inline

df = pd.read_csv("twcs/twcs.csv")
df.sort_values(by='tweet_id', inplace=True)
df.head(20)

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0
5,6,sprintcare,False,Tue Oct 31 21:46:24 +0000 2017,@115712 Can you please send us a private messa...,"5,7",8.0
2314173,7,115712,True,Tue Oct 31 21:47:48 +0000 2017,@sprintcare the only way I can get a response ...,NaN,6.0
6,8,115712,True,Tue Oct 31 21:45:10 +0000 2017,@sprintcare is the worst customer service,"9,6,10",NaN
2314166,9,sprintcare,False,Tue Oct 31 21:46:14 +0000 2017,@115712 I would love the chance to review the ...,NaN,8.0
2314174,10,sprintcare,False,Tue Oct 31 21:45:59 +0000 2017,@115712 Hello! We never like our customers to ...,NaN,8.0


In [45]:
"""
  Data preprocessing
"""

chat_words = {
    "afaik": "as far as i know",
    "asap": "as soon as possible",
    "atk": "at the keyboard",
    "atm": "at the moment",
    "brb": "be right back",
    "brt": "be right there",
    "btw": "by the way",
    "b4": "before",
    "cu": "see you",
    "cya": "see you",
    "faq": "frequently asked questions",
    "fc": "fingers crossed",
    "fyi": "for your information",
    "gn": "good night",
    "gr8": "great",
    "g9": "genius",
    "ic": "i see",
    "imo": "in my opinion",
    "iow": "in other words",
    "lol": "laughing out loud",
    "l8r": "later",
    "mte": "my thoughts rxactly",
    "m8": "mate",
    "nrn": "no reply necessary",
    "oic": "oh i see",
    "rofl": "rolling on the floor laughing",
    "thx": "thank you",
    "ttyl": "talk to you later",
    "u": "you",
    "u2": "you too",
    "w8": "wait",
    "imma": "i am going to",
    "2nite": "tonight",
    "dmed": "mesaged",
    'dm': "message"
}

contractions = {
    "ain't": "are not",
    "aren't": "are not",
    "'bout": "about",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "here's": "here is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "I would",
    "i'd've": "I would have",
    "i'll": "I will",
    "i'll've": "I will have",
    "i'm": "I am",
    "i've": "I have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "i phone": "iphone",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so is",
    "that'd": "that had",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they're": "they are",
    "they've": "they have",
    "'til": "until",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'll": "you will",
    "you're": "you are",
    "you've": "you have",
}

punc = string.punctuation

def remove_punctuations(text): 
    return text.translate(str.maketrans("", "", punc))

def remove_chat_words(text):
    new_text = []
    for word in text.split(' '):
        if word in chat_words.keys():
            new_text += chat_words[word].split(' ')
        else:
            new_text.append(word)
    return ' '.join(new_text)

def remove_contractions(text):
    new_text = []
    for word in text.split(' '):
        if word in contractions.keys():
            new_text += contractions[word].split(' ')
        else:
            new_text.append(word)
    return ' '.join(new_text)

def remove_urls(text):
    return re.sub(r'https://t\.co/\w+', '<url>', text)

def remove_mentions(text):
    return re.sub(r'@[0-9A-Za-z_\-]+', '<user>', text)

def remove_hashtags(text):
    return re.sub(r'#[0-9A-Za-z_\-]+', '', text)

def remove_emojis(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def clean(text):
    text = re.sub(r'[“”]', '"', text)
    text = re.sub(r"’", "'", text)
    text = text.lower()
    text = remove_chat_words(text)
    text = remove_contractions(text)
    text = remove_urls(text)
    text = remove_mentions(text)
    text = remove_hashtags(text)
    text = re.sub(r'&gt;', '>', text)
    text = re.sub(r'&lt;', '<', text)
    #text = remove_punctuations(text) # da li treba
    #text = remove_stopwords(text)
    #text = remove_emojis(text)
    #text = remove_emoticons(text)
    return text

def id2text(df, x):
    row = df[df.tweet_id == int(x)]
    return '' if (len(row) == 0) else clean(row.iloc[0]['text'])

apple_support.value_counts()
apple_support = df[df.author_id == "AppleSupport"]
apple_support = apple_support[~apple_support.in_response_to_tweet_id.isna()]
apple_support.value_counts()
apple_support = apple_support[['author_id', 'text', 'in_response_to_tweet_id']]
apple_support.loc[:, 'in_response_to_tweet_id'] = apple_support.in_response_to_tweet_id.apply(lambda x: id2text(df, x))
apple_support.loc[:, 'text'] = apple_support.text.apply(lambda x: clean(x))

apple_support.head(100)

,author_id,text,in_response_to_tweet_id
396,AppleSupport,<user> we are here for you. which version of t...,<user> <url>
398,AppleSupport,<user> lets take a closer look into this issue...,<user> the newest update. i️ made sure to down...
401,AppleSupport,<user> let us go to message for the next steps...,<user> tried resetting my settings .. restarti...
403,AppleSupport,<user> any steps tried since it started last n...,<user> this is what it looks like <url>
405,AppleSupport,<user> that is great it has ios 11.1 as we can...,<user> i️ have an iphone 7 plus and yes i️ do
...,...,...,...
3594,AppleSupport,<user> are all of your apps updated as well?,<user> ios 11.1
3596,AppleSupport,<user> we would be happy to look into this wit...,<user> it
3599,AppleSupport,<user> we know how important it is to be able ...,<user> whenever i try to see the photos that I...
3602,AppleSupport,<user> we appreciate those details. for more h...,<user> and the new ones tauntingly pretend the...
